In [33]:
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import datetime
import numpy as np
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, DIFFICULTIES, PROBLEM_INDICES
from itertools import product
from IPython.display import display, HTML
from util.scrape_log import scrape_search_log
from pathlib import Path

In [34]:
_GENPLAN_PAPER_DIR=str(Path.home())+"/papers/manuscripts/genplan-23-kernels"
_ICAPS_PAPER_DIR=str(Path.home())+"/papers/manuscripts/icaps-24-kernels"
pd.options.display.float_format = "{:.1f}".format

In [35]:
## collect logs
# os.system("sh scripts_gnn/collect_cluster1_test.sh")
# os.system("sh scripts_kernel/collect_gadi_test.sh")

In [36]:
_MODELS = [
    "blind",
    "hff",
    "lama-first",
    "muninn",
    "goose-genplan23",
    #   "gL4max",
    #   "gL4mean",
    "gI4max",
    "gI4mean",
    "1wl",
    # #   "1wl_1_linear-svr",
    # "1wl_4_linear-svr",
    # #   "1wl_4_gp",
    # #   "2gwl_1_linear-svr",
    # "2gwl_4_linear-svr",
    # #   "2lwl_1_linear-svr",
    # "2lwl_4_linear-svr",
]

NEW = {
    "1wl",  # 1wl 4 iterations after bug fix with weights
}

WLS = {
    "1wl",
    # below are old
    "1wl_1_linear-svr",
    "2gwl_1_linear-svr",
    "2lwl_1_linear-svr",
    "1wl_4_linear-svr",
    "1wl_4_gp",
    "2gwl_4_linear-svr",
    "2lwl_4_linear-svr",
}
CLASSICAL = {
    "blind",
    "hff",
    "lama-first",
}
GNNS = {
    "muninn",
    "goose-genplan23",
    "gL4max",
    "gL4mean",
    "gI4max",
    "gI4mean",
}

PLOT_MAX_EXPANDED = 1e8
PLOT_MAX_COST = 1e4

In [37]:
RENAME = {
    "goose-genplan23": "gGenPlan23",
}


def rename(model):
    if model in RENAME:
        return RENAME[model]
    model = model.replace("_linear-svr", "_svr")
    model = model.replace("_gp", "_gpr")
    return model


def marker_style(model):
    if model in CLASSICAL:
        return "o"
    if model in WLS:
        return "x"
    if model in GNNS:
        return "+"
    raise ValueError(model)

In [38]:
with open("ipc-2023-learning-reference-costs.pkl", 'rb') as file:
    REFERENCE_COSTS = {domain : [] for domain in IPC2023_LEARNING_DOMAINS}
    for (domain, diff_in_int, problem_index), reference_cost in pickle.load(file).items():
        REFERENCE_COSTS[domain].append(reference_cost)
## (domain) -> [reference_costs_1, ..., reference_costs_90]
# REFERENCE_COSTS

In [39]:
""" Preprocess data """
def get_data_tmp(domain):
    data = {}

    for model in _MODELS:
        d = {
            "solved": [],
            "expanded": [],
            "cost": [],
            "first_h": [],
            "quality": [],
            "c_opt": [],
            "c_opt_min": [],
            "difficulty": [],
            "time": [],
            "tried": [],
        }

        for diff_i, difficulty in enumerate(DIFFICULTIES):
            for prob_i, pf_name in enumerate(PROBLEM_INDICES):
                pf = f"../benchmarks/ipc2023-learning-benchmarks/{domain}/testing/{difficulty}/{pf_name}.pddl"
                assert os.path.exists(pf)
                stats = None
                if model=="muninn":
                    stats = {
                        "first_h": -1,
                        "solved": 0,
                        "time": -1,
                        "cost": -1,
                        "expanded": -1,
                        "evaluated": -1,
                        "tried": 1,
                    }
                    log_file = f"logs/muninn/{domain}_{difficulty}_{pf_name}.log"
                    plan_file = f"logs/muninn/{domain}_{difficulty}_{pf_name}.plan"
                    cost = 0
                    if os.path.exists(plan_file):
                        with open(plan_file, "r") as f:
                            for line in f.readlines():
                                if len(line) > 0 and line[0] == "(":
                                    cost += 1
                    if cost > 0:
                        stats["solved"] = 1
                        stats["cost"] = cost
                    for line in open(log_file, 'r').readlines():
                        if "[1]" in line and stats['solved'] == 0:
                            toks = line.replace(",", "").replace("(", "").split()
                            stats["first_h"] = float(toks[4])
                        elif "Walltime Used:" in line:
                            time_object = datetime.datetime.strptime(line.split()[-1], "%H:%M:%S")
                            stats["time"] = time_object.hour * 3600 + time_object.minute * 60 + time_object.second


                elif model == "goose-genplan23":
                    log_file = f"icaps24_test_logs_old/{domain}_{difficulty}_{pf_name}_{model}.log"
                elif model[0] == "g":
                    # gI4max
                    rep = {"I": "ilg", "L": "llg"}[model[1]]
                    L = model[2]
                    aggr = model[3:]
                    log_file = f"icaps24_test_logs_old/{domain}_{difficulty}_{pf_name}_{domain}_{rep}_L{L}_H64_{aggr}_r0.log"
                elif model in WLS:
                    log_dir = "icaps24_test_logs"
                    if model not in NEW:
                        log_dir += "_old"
                        desc = model
                    else:
                        desc = model + "_4_linear-svr"
                    log_file = f"{log_dir}/{domain}_{difficulty}_{pf_name}_ilg_{desc}_H.log"
                else:
                    assert model in CLASSICAL
                    log_file = f"logs/ipc2023_{model}/{domain}_{difficulty}_{pf_name}_{model}.log"

                if stats is None:
                    stats = scrape_search_log(log_file)

                if stats["time"] >= 1800:
                    stats["solved"] = 0
                    stats["time"] = 1800

                c = stats["cost"] if stats["solved"] else -1
                c_opt = REFERENCE_COSTS[domain][diff_i * 30 + prob_i]
                quality = c_opt / c
                quality = max(quality, 0)
                quality = min(quality, 1)
                stats["quality"]=(quality)
                stats["c_opt"]=(c_opt)
                stats["c_opt_min"]=min(c_opt, stats["cost"])
                stats["difficulty"]=(difficulty)

                for k in d:
                    d[k].append(stats[k] if k in stats else -1)

        data[model] = d

    return data

data_domain_helper = {}
for domain in IPC2023_LEARNING_DOMAINS:
    data_domain_helper[domain] = get_data_tmp(domain)

def get_data(domain):
    return data_domain_helper[domain]

def get_whole_df():
    ret = {"domain": [], "model": [], "problem": []}
    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        stat_keys = set(data[_MODELS[0]].keys()).difference({"difficulty"})
        for k in stat_keys:
            if k in ret:
                continue
            ret[k] = []
        for model in _MODELS:
            for i in range(90):
                ret["domain"].append(domain)
                ret["model"].append(rename(model))
                ret["problem"].append(i+1)
                for k in stat_keys:
                    ret[k].append(data[model][k][i])
    return pd.DataFrame(data=ret)
WHOLE_DF = get_whole_df()
# WHOLE_DF

In [40]:
df = WHOLE_DF
df = df[df["solved"] == True]
df = df[df["model"] == "1wl"]
fig = px.scatter(df, x="c_opt_min", y="first_h", color="domain", log_x=True, log_y=True)
max_metric= 1e4
diagonal = px.line(x=[1, max_metric], y=[1, max_metric], color_discrete_sequence=['gray'])
fig = go.Figure(diagonal.data + fig.data, layout=fig.layout)
fig.show()

## Summary of scores

In [58]:
def get_table_df(models=None, metric="quality"):
    d = {
        "domain": [],
    }
    if models is None:
        models = _MODELS
    for config in models:
        d[rename(config)] = []

    sum_scores = {config: 0 for config in models}

    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        d["domain"].append(domain)

        for config in models:
            score = sum(data[config][metric])
            d[rename(config)].append(score)
            sum_scores[config] += score

    d["domain"].append("sum")
    for config in models:
        d[rename(config)].append(sum_scores[config])

    return pd.DataFrame(d)

In [42]:
print("TODO: find the floortile goose can solve")

for metric in [
    "solved",
    "quality",
]:
    scores_df = get_table_df(metric=metric)
    scores = scores_df.to_numpy()
    scores = scores[:, 1:]
    display(scores_df)

TODO: find the floortile goose can solve


,domain,blind,hff,lama-first,muninn,gGenPlan23,gI4max,gI4mean,1wl
0,blocksworld,8,28,61,40,29,49,58,49
1,childsnack,9,26,35,11,10,19,20,20
2,ferry,10,68,68,46,60,64,72,74
3,floortile,2,12,11,0,1,0,0,2
4,miconic,30,90,90,30,73,90,90,90
5,rovers,15,34,67,15,20,25,29,45
6,satellite,12,65,89,18,29,31,29,37
7,sokoban,27,36,40,26,30,32,33,37
8,spanner,30,30,30,32,30,30,33,30
9,transport,9,41,66,17,31,38,35,49


,domain,blind,hff,lama-first,muninn,gGenPlan23,gI4max,gI4mean,1wl
0,blocksworld,8.0,14.1,39.8,40.0,23.9,45.6,52.7,44.4
1,childsnack,9.0,20.1,22.0,11.0,8.7,17.6,19.9,18.9
2,ferry,10.0,67.6,62.9,46.0,59.7,63.9,71.9,73.6
3,floortile,2.0,11.2,10.0,0.0,0.9,0.0,0.0,1.8
4,miconic,30.0,88.5,81.6,30.0,72.0,89.2,89.1,89.0
5,rovers,15.0,32.7,65.2,14.2,17.7,19.7,24.8,36.4
6,satellite,12.0,63.8,87.3,18.0,24.1,24.6,21.6,32.9
7,sokoban,27.0,26.3,29.8,24.3,26.7,28.3,30.0,33.1
8,spanner,30.0,30.0,30.0,32.0,29.6,30.0,33.0,27.6
9,transport,9.0,39.3,64.0,17.0,29.0,35.6,31.6,46.3


## Plots

In [43]:
def plotall(metric, models, plot_reference_cost=True, log_yscale=True, ylim=None):
    save_dir = f"px/{metric}"
    os.makedirs(save_dir, exist_ok=True)

    df = WHOLE_DF
    df = df[df['model'].isin(models)]
    for domain in IPC2023_LEARNING_DOMAINS:
        df_domain = df[df['domain'] == domain]
        fig = px.scatter(df_domain, x="problem", y=metric, title=domain, color="model", log_y=log_yscale, range_y=ylim, range_x=(0, 90))
        
        if plot_reference_cost:
            # otherwise, will loop the line around
            df_line = df_domain[df_domain['model'] == models[0]]
            reference_line = px.line(df_line, x="problem", y="c_opt", color_discrete_sequence=['black'], log_y=log_yscale)
            fig = go.Figure(data=fig.data + reference_line.data, layout=fig.layout)

        # fig.show()
        fig.write_html(f"{save_dir}/{domain}.html")
        plt.clf()

In [44]:
def plot_versus(model_x, model_y, metric):
    if metric == "expanded":
        max_metric = PLOT_MAX_EXPANDED
        log_scale = True
    elif metric == "cost":
        max_metric = PLOT_MAX_COST
        log_scale = True
    else:
        raise NotImplementedError(metric)
    
    save_dir = f"px/versus_{metric}"
    os.makedirs(save_dir, exist_ok=True)

    d = {"domain": [], model_x: [], model_y: []}
    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        for i in range(90):
            for model in [model_x, model_y]:
                if i >= len(data[model]["solved"]) or not data[model]["solved"][i]:
                    val = max_metric
                else:
                    val = data[model][metric][i]
                d[model].append(val)
            d["domain"].append(domain)
    df = pd.DataFrame(data=d)
    diagonal = px.line(x=[1, max_metric], y=[1, max_metric], color_discrete_sequence=['gray'])
    h = px.line(x=[max_metric, max_metric], y=[1, max_metric], color_discrete_sequence=['gray'])
    v = px.line(x=[1, max_metric], y=[max_metric, max_metric], color_discrete_sequence=['gray'])
    scatter = px.scatter(df, x=model_x, y=model_y, color="domain", log_x=log_scale, log_y=log_scale, range_x=(log_scale, max_metric*1.1), range_y=(log_scale, max_metric*1.1))
    fig = go.Figure(diagonal.data + h.data + v.data + scatter.data, layout=scatter.layout)
    fig.write_html(f"{save_dir}/{model_x}_vs_{model_y}.html")

### Versus

In [45]:
metrics = ["expanded", "cost"]
pairs = [
    ("gI4max", "gI4mean"),
    # ("1wl_1", "2lwl_1"),
    ("1wl", "gI4mean"),
    # ("1wl_1", "1wl_4"),
    # ("2lwl_1", "2lwl_4"),
    ("goose-genplan23", "gI4mean"),
]
for metric, (mx, my) in product(metrics, pairs):
    plot_versus(model_x=mx, model_y=my, metric=metric)

### Expanded 

In [46]:
plotall(
    "expanded",
    ylim=(1, PLOT_MAX_EXPANDED),
    models=[
        "blind",
        "hff",
        "goose-genplan23",
        "gI4max",
        "gI4mean",
        "1wl",
        # # "1wl_1",
        # "1wl_4",
        # # "2gwl_1",
        # "2gwl_4",
        # # "2lwl_1",
        # "2lwl_4",
    ],
)

<Figure size 640x480 with 0 Axes>

### Plan quality


In [47]:
plotall(
    "cost",
    ylim=(1, PLOT_MAX_COST),
    models=[
        "blind",
        "hff",
        "muninn",
        "goose-genplan23",
        "gI4max",
        "gI4mean",
        "1wl",
        # # "1wl_1",
        # "1wl_4",
        # # "2gwl_1",
        # "2gwl_4",
        # # "2lwl_1",
        # "2lwl_4",
    ],
)

<Figure size 640x480 with 0 Axes>

### Cumulative runtime

In [48]:
fig = px.ecdf(
    WHOLE_DF,
    x="time",
    y="solved",
    color="model",
    ecdfnorm=None,
    log_x=True,
    range_x=(1, 2000),
)
fig.show()
fig.write_html("px/cumulative_runtime.html")

### Cumulative expanded

In [49]:
exp_df = WHOLE_DF
exp_df["expanded"] = exp_df["expanded"] * exp_df["solved"]
fig = px.ecdf(exp_df, x="expanded", y="solved", color="model", ecdfnorm=None, log_x=True)
fig.show()
fig.write_html("px/cumulative_expanded.html")

### Hit WL colours

In [50]:
ITRS = 4
ret = {"domain": [], "model": [], "problem": [], "ratio": [], "itr": []}

model = "1wl"  # TODO if we run other wl turn this into a loop
for domain in IPC2023_LEARNING_DOMAINS:
    for diff_i, difficulty in enumerate(DIFFICULTIES):
        for prob_i, pf_name in enumerate(PROBLEM_INDICES):
            pf = f"../benchmarks/ipc2023-learning-benchmarks/{domain}/testing/{difficulty}/{pf_name}.pddl"
            assert os.path.exists(pf)
            desc = f"{model}_{ITRS}_linear-svr"  # TODO generalise
            log_file = f"icaps24_test_logs/{domain}_{difficulty}_{pf_name}_ilg_{desc}_H.log"

            if not os.path.exists(log_file):
                continue

            tmp = {}

            for line in open(log_file, "r").readlines():

                if "seen/unseen colours in itr" in line:
                    line = line.replace(":", "")
                    toks = line.split()
                    itr = int(toks[-3])
                    seen = int(toks[-2])
                    unseen = int(toks[-1])
                    ratio = seen / (seen + unseen)
                    tmp[itr] = ratio
                
            if len(tmp) == ITRS:
                for itr, ratio in tmp.items():
                    ret["domain"].append(domain)
                    ret["model"].append(model)
                    ret["problem"].append(30 * diff_i + prob_i)
                    ret["ratio"].append(ratio)
                    ret["itr"].append(itr+1)

RATIO_DF = pd.DataFrame(data=ret)

In [51]:
def plot_ratio(model):
    save_dir = f"px/ratio"
    os.makedirs(save_dir, exist_ok=True)

    df = RATIO_DF.copy()
    df = df[df['model']==model]
    for domain in IPC2023_LEARNING_DOMAINS:
        df_domain = df[df['domain'] == domain]
        fig = px.scatter(df_domain, x="problem", y="ratio", title=f"{model}_{domain}", color="itr", range_y=(0, 1), range_x=(0, 90))
        
        # fig.show()
        fig.write_html(f"{save_dir}/{model}_{domain}.html")
        plt.clf()

In [52]:
plot_ratio(model="1wl")

<Figure size 640x480 with 0 Axes>

## More refined data

In [60]:
def get_metrics_df(data, models=None):
  d = {
    "config": [],
    "quality": [],
    "solved": [],
    "tried": [],
    "e_quality": [],
    "e_solved": [],
    "e_tried": [],
    "m_quality": [],
    "m_solved": [],
    "m_tried": [],
    "h_quality": [],
    "h_solved": [],
    "h_tried": [],
  }
  if models is None:
    models = _MODELS
  for config in models:
    d["config"].append(config)
    d["quality"].append(sum(data[config]["quality"]))
    d["solved"].append(sum(data[config]["solved"]))
    d["tried"].append(len(data[config]["solved"]))
    for difficulty in DIFFICULTIES:
      quality = 0
      solved = 0
      tried = 0
      for i, prob_solved in enumerate(data[config]["solved"]):
        if data[config]["difficulty"][i] == difficulty:
          quality += data[config]["quality"][i]
          solved += prob_solved
          tried += data[config]["tried"][i]
      d[f"{difficulty[0]}_quality"].append(quality)
      d[f"{difficulty[0]}_solved"].append(solved)
      d[f"{difficulty[0]}_tried"].append(tried)
  return pd.DataFrame(d)

In [54]:
for domain in IPC2023_LEARNING_DOMAINS:
    print(domain)
    df = get_metrics_df(get_data(domain))
    display(df)

blocksworld


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,8.0,8,90,8.0,8,30,0.0,0,30,0.0,0,30
1,hff,14.1,28,90,14.1,28,30,0.0,0,30,0.0,0,30
2,lama-first,39.8,61,90,20.8,30,30,17.1,27,30,1.9,4,30
3,muninn,40.0,40,90,30.0,30,30,10.0,10,30,0.0,0,30
4,goose-genplan23,23.9,29,90,23.0,28,30,0.9,1,30,0.0,0,30
5,gI4max,45.6,49,90,28.1,30,30,14.8,16,30,2.7,3,18
6,gI4mean,52.7,58,90,27.2,30,30,21.9,24,30,3.6,4,20
7,1wl,44.4,49,90,26.7,30,30,17.8,19,30,0.0,0,30


childsnack


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,9.0,9,90,9.0,9,30,0.0,0,30,0.0,0,30
1,hff,20.1,26,90,20.1,26,30,0.0,0,30,0.0,0,30
2,lama-first,22.0,35,90,18.5,29,30,3.0,5,30,0.6,1,30
3,muninn,11.0,11,90,11.0,11,30,0.0,0,30,0.0,0,30
4,goose-genplan23,8.7,10,90,8.7,10,30,0.0,0,30,0.0,0,30
5,gI4max,17.6,19,90,17.6,19,30,0.0,0,3,0.0,0,0
6,gI4mean,19.9,20,90,19.9,20,28,0.0,0,0,0.0,0,0
7,1wl,18.9,20,90,18.9,20,30,0.0,0,30,0.0,0,0


ferry


...

,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,27.0,27,90,27.0,27,30,0.0,0,30,0,0,30
1,hff,26.3,36,90,23.2,30,30,3.1,6,30,0,0,30
2,lama-first,29.8,40,90,22.4,30,30,7.4,10,30,0,0,30
3,muninn,24.3,26,90,24.3,26,30,0.0,0,30,0,0,30
4,goose-genplan23,26.7,30,90,25.7,29,30,1.0,1,30,0,0,30
5,gI4max,28.3,32,90,26.3,30,30,2.0,2,18,0,0,0
6,gI4mean,30.0,33,90,27.0,30,30,3.0,3,23,0,0,0
7,1wl,33.1,37,90,26.1,30,30,7.0,7,30,0,0,0


spanner


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,30.0,30,90,30.0,30,30,0,0,30,0,0,30
1,hff,30.0,30,90,30.0,30,30,0,0,30,0,0,30
2,lama-first,30.0,30,90,30.0,30,30,0,0,30,0,0,30
3,muninn,32.0,32,90,30.0,30,30,2,2,30,0,0,30
4,goose-genplan23,29.6,30,90,29.6,30,30,0,0,30,0,0,30
5,gI4max,30.0,30,90,30.0,30,30,0,0,13,0,0,0
6,gI4mean,33.0,33,90,30.0,30,30,3,3,21,0,0,0
7,1wl,27.6,30,90,27.6,30,30,0,0,30,0,0,0


transport


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,9.0,9,90,9.0,9,30,0.0,0,30,0,0,30
1,hff,39.3,41,90,28.3,30,30,11.0,11,30,0,0,30
2,lama-first,64.0,66,90,28.0,30,30,30.0,30,30,6,6,30
3,muninn,17.0,17,90,17.0,17,30,0.0,0,30,0,0,30
4,goose-genplan23,29.0,31,90,27.0,29,30,2.0,2,30,0,0,30
5,gI4max,35.6,38,90,27.9,30,30,7.8,8,17,0,0,0
6,gI4mean,31.6,35,90,27.5,30,30,4.1,5,26,0,0,0
7,1wl,46.3,49,90,28.3,30,30,18.0,19,30,0,0,0


## To GenPlan23 latex table

In [69]:
GENPLAN_MODELS = {
    "blind": "blind",
    "hff": "hff",
    "muninn": "muninn",
    "gI4max": "goosemax",
    "gI4mean": "goosemean",
    "1wl": "wl",
}

In [76]:
""" coverage_tall.tex and quality_tall.tex """

coverage_i = [5, 8, 11, 2]
quality_i = [4, 7, 10, 1]

for metric, indices in [("coverage", coverage_i), ("quality", quality_i)]:
    nnn = 0
    genplan_models_list = list(GENPLAN_MODELS.keys())
    scores_df = get_table_df(models=genplan_models_list)
    scores = scores_df.to_numpy()
    scores = scores[:,1:]

    ret  = r"\begin{tabular}{" + "c " + " ".join(["p{\coveragewidth}" for _ in range(len(GENPLAN_MODELS))]) + r"} \toprule "
    # ret  = r"\begin{tabular}{" + "c " + " ".join(["c" for _ in range(len(GENPLAN_MODELS))]) + r"} \toprule Domain "
    ret += r" & & & &\multicolumn{3}{c}{GOOSE} \\ \cmidrule(lr){5-7} Domain "

    for model in GENPLAN_MODELS.values():
        ret += f"& \\header{{\{model.replace('-', '')}}} "
    ret += r"\\ \midrule" + '\n'

    all_solved = {
    "easy": np.zeros(len(GENPLAN_MODELS)).astype(int),
    "medium": np.zeros(len(GENPLAN_MODELS)).astype(int),
    "hard": np.zeros(len(GENPLAN_MODELS)).astype(int),
    "all": np.zeros(len(GENPLAN_MODELS)).astype(int),
    }
    all_quality = {
    "easy": np.zeros(len(GENPLAN_MODELS)),
    "medium": np.zeros(len(GENPLAN_MODELS)),
    "hard": np.zeros(len(GENPLAN_MODELS)),
    "all": np.zeros(len(GENPLAN_MODELS)),
    }

    for domain in IPC2023_LEARNING_DOMAINS:
        ret += f"\\multirow{{4}}{{*}}{{{domain}}}"
        data = (get_metrics_df(get_data(domain), models = genplan_models_list).to_numpy())

        # coverage then cost
        solved = {
            "easy": data[:,indices[0]].T,
            "medium": data[:,indices[1]].T,
            "hard": data[:,indices[2]].T,
            "all": data[:,indices[3]].T,
        }

        for difficulty in ["easy", "medium", "hard", "all"]:
            scores_domain = solved[difficulty]
            sorted_scores_domain = sorted(scores_domain, reverse=True)
            all_solved[difficulty] += np.array(scores_domain).astype(int)
            for i in range(len(solved["easy"])):
                chunks = []
                best_score = max(scores_domain)
                score = solved[difficulty][i]
                if metric == "quality":
                    display_score = f"{score:.1f}"
                else:
                    display_score = score
                if score == 0:
                    chunks.append("-")
                elif score==sorted_scores_domain[0]:
                    chunks.append(f"\\first{{{display_score}}}")
                elif score==sorted_scores_domain[1]:
                    chunks.append(f"\\second{{{display_score}}}")
                elif score==sorted_scores_domain[2]:
                    chunks.append(f"\\third{{{display_score}}}")
                else:
                    chunks.append(f"{display_score}")

                chunk = " & ".join(chunks)
                ret += f" & {chunk}"

            if difficulty != "all":
                ret += "\\\\ \n"

        if domain==IPC2023_LEARNING_DOMAINS[-1]:
            ret += "\\\\ \midrule \n"
        else:
            ret += "\\\\ \cmidrule(lr){2-" + str(len(GENPLAN_MODELS)+1) + "} \n"

    ret += f"\\multirow{{4}}{{*}}{{sum}}"

    for difficulty in ["easy", "medium", "hard", "all"]:
        for i in range(len(all_solved["easy"])):
            chunks = []

            scores_domain = all_solved[difficulty]

            sorted_scores_domain = sorted(scores_domain, reverse=True)
            sorted_scores_domain = [round(x, 1) for x in sorted_scores_domain]
            best_score = max(scores_domain)
            score = all_solved[difficulty][i]
            if metric == "quality":
                display_score = f"{score:.1f}"
            else:
                display_score = score
            if score == 0:
                chunks.append("-")
            elif score==sorted_scores_domain[0]:
                chunks.append(f"\\first{{{display_score}}}")
            elif score==sorted_scores_domain[1]:
                chunks.append(f"\\second{{{display_score}}}")
            elif score==sorted_scores_domain[2]:
                chunks.append(f"\\third{{{display_score}}}")
            else:
                chunks.append(f"{display_score}")

            chunk = " & ".join(chunks)
            ret += f" & {chunk}"

        if difficulty != "all":
            ret += "\\\\ \n"

    ret += r" \\ \bottomrule\end{tabular}"

    f = open(f"{_GENPLAN_PAPER_DIR}/figures/{metric}_tall.tex", 'w')
    f.write(ret)
    f.close()

['blind', 'hff', 'muninn', 'gI4max', 'gI4mean', '1wl']
['blind', 'hff', 'muninn', 'gI4max', 'gI4mean', '1wl']


# To ICAPS24 latex table

In [57]:
""" coverage_tall.tex """
nnn = 0

scores_df = get_table_df()
scores = scores_df.to_numpy()
scores = scores[:,1:]

# ret  = r"\begin{tabular}{ c " + " ".join(["c" for _ in range(4*len(_MODELS))]) + r"} \toprule Domain "
ret  = r"\begin{tabular}{" + "c " + " ".join(["p{\coveragewidth}" for _ in range(len(_MODELS))]) + r"} \toprule Domain "

for model in _MODELS:
    ret += f"& \\header{{\{model.replace('-', '')}}} "
ret += r"\\ \midrule" + '\n'

all_solved = {
  "easy": np.zeros(len(_MODELS)).astype(int),
  "medium": np.zeros(len(_MODELS)).astype(int),
  "hard": np.zeros(len(_MODELS)).astype(int),
  "all": np.zeros(len(_MODELS)).astype(int),
}
all_quality = {
  "easy": np.zeros(len(_MODELS)),
  "medium": np.zeros(len(_MODELS)),
  "hard": np.zeros(len(_MODELS)),
  "all": np.zeros(len(_MODELS)),
}

for domain in IPC2023_LEARNING_DOMAINS:
    ret += f"\\multirow{{4}}{{*}}{{{domain}}}"
    data = (get_metrics_df(get_data(domain)).to_numpy())

    # coverage then cost
    solved = {
        "easy": data[:,5].T,
        "medium": data[:,8].T,
        "hard": data[:,11].T,
        "all": data[:,2].T,
    }

    for difficulty in ["easy", "medium", "hard", "all"]:
        scores_domain = solved[difficulty]
        sorted_scores_domain = sorted(scores_domain, reverse=True)
        all_solved[difficulty] += np.array(scores_domain).astype(int)
        for i in range(len(solved["easy"])):
            chunks = []
            best_score = max(scores_domain)
            score = solved[difficulty][i]
            if score == 0:
                chunks.append("-")
            elif score==sorted_scores_domain[0]:
                chunks.append(f"\\first{{{score}}}")
            elif score==sorted_scores_domain[1]:
                chunks.append(f"\\second{{{score}}}")
            elif score==sorted_scores_domain[2]:
                chunks.append(f"\\third{{{score}}}")
            else:
                chunks.append(f"{score}")

            chunk = " & ".join(chunks)
            ret += f" & {chunk}"

        if difficulty != "all":
            ret += "\\\\ \n"

    if domain==IPC2023_LEARNING_DOMAINS[-1]:
        ret += "\\\\ \midrule \n"
    else:
        ret += "\\\\ \cmidrule(lr){2-" + str(len(_MODELS)+1) + "} \n"

ret += f"\\multirow{{4}}{{*}}{{sum}}"

for difficulty in ["easy", "medium", "hard", "all"]:
    for i in range(len(all_solved["easy"])):
        chunks = []

        scores_domain = all_solved[difficulty]

        sorted_scores_domain = sorted(scores_domain, reverse=True)
        sorted_scores_domain = [round(x, 1) for x in sorted_scores_domain]
        best_score = max(scores_domain)
        score = all_solved[difficulty][i]
        if score == 0:
            chunks.append("-")
        elif score==sorted_scores_domain[0]:
            chunks.append(f"\\first{{{score}}}")
        elif score==sorted_scores_domain[1]:
            chunks.append(f"\\second{{{score}}}")
        elif score==sorted_scores_domain[2]:
            chunks.append(f"\\third{{{score}}}")
        else:
            chunks.append(f"{score}")

        chunk = " & ".join(chunks)
        ret += f" & {chunk}"

    if difficulty != "all":
        ret += "\\\\ \n"

ret += r" \\ \bottomrule\end{tabular}"

f = open(f"{_ICAPS_PAPER_DIR}/figures/coverage_tall.tex", 'w')
f.write(ret)
f.close()